<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bo-YAN---HW-1-Question-1" data-toc-modified-id="Bo-YAN---HW-1-Question-1-1">Bo YAN - HW 1 Question 1</a></span><ul class="toc-item"><li><span><a href="#Using-the-Patentholders.csv-data-set,-find-names-that-match-the-following-strings" data-toc-modified-id="Using-the-Patentholders.csv-data-set,-find-names-that-match-the-following-strings-1.1">Using the Patentholders.csv data set, find names that match the following strings</a></span></li></ul></li><li><span><a href="#1.-Create-functions-and-decalrations" data-toc-modified-id="1.-Create-functions-and-decalrations-2">1. Create functions and decalrations</a></span><ul class="toc-item"><li><span><a href="#1.1---Functions" data-toc-modified-id="1.1---Functions-2.1">1.1 - Functions</a></span></li><li><span><a href="#1.2---Declarations" data-toc-modified-id="1.2---Declarations-2.2">1.2 - Declarations</a></span></li></ul></li><li><span><a href="#2(a)-&quot;United-States-Navy&quot;" data-toc-modified-id="2(a)-&quot;United-States-Navy&quot;-3">2(a) "United States Navy"</a></span></li><li><span><a href="#2(b)-&quot;IBM&quot;" data-toc-modified-id="2(b)-&quot;IBM&quot;-4">2(b) "IBM"</a></span></li><li><span><a href="#2(c)-&quot;Univ.-of-Calif.&quot;" data-toc-modified-id="2(c)-&quot;Univ.-of-Calif.&quot;-5">2(c) "Univ. of Calif."</a></span></li></ul></div>

# Bo YAN - HW 1 Question 1
## Using the Patentholders.csv data set, find names that match the following strings

(a) "United States Navy"

(b) "IBM"

(c) "Univ. of Calif."


In [1]:
import py_stringmatching as sm 
import pandas as pd
import numpy as np 
from sklearn.metrics import precision_score, recall_score
from collections import Counter
import re

# 1. Create functions and decalrations

## 1.1 - Functions

In [2]:
def load_data(data_source, column_name):
    df_name = pd.read_csv(data_source, header = None, names = [column_name])
    df_name.drop_duplicates(subset = column_name, inplace = True)
    return df_name

In [3]:
def pruning_by_string(data, column_name, string):
    data = data[data[column_name].str.contains(string, case = False)]
    data.reset_index(drop = True, inplace = True)
    return data

In [4]:
def get_tokens(data, column_name):
    patent_hoders = data[column_name].to_list()
    ids = data.index.to_list()
    id2gid = {}
    id2wstoken = {}
    id2qg3tokens = {}
    for i in range(len(patent_hoders)):
        id2wstoken[ids[i]] = ws_tok.tokenize(patent_hoders[i])
        id2qg3tokens[ids[i]] = qg3_tok.tokenize(patent_hoders[i])    
    return id2wstoken, id2qg3tokens, ids, id2gid

In [5]:
def match_values_to_group(id2wstoken, index_list, similarity_measure, threshould):
    id2pgid = {}
    pgid2ids = {}
    crt_gid = 0
    # for each patent holder, if it matches with any patent holder, put them in the same group 
    for crt_id in index_list:
        max_sim = 0
        best_gid = crt_gid
        crt_token = id2wstoken[crt_id]
        for processed_id in id2pgid:
            processed_token = id2wstoken[processed_id]
            sim = similarity_measure.get_sim_score(processed_token, crt_token)
            if sim > max_sim:
                max_sim = sim
                best_gid = id2pgid[processed_id]
        if max_sim > threshould:
            id2pgid[crt_id] = best_gid
            pgid2ids[best_gid].append(crt_id)
        else:
            id2pgid[crt_id] = crt_gid
            pgid2ids[crt_gid] = [crt_id]
            crt_gid = crt_gid + 1    
    
    return pgid2ids

In [6]:
def find_matching_values(data, pgid2ids, target_string, threshould, word_token):
    final_list = []
    for i in range(len(pgid2ids)):
        for p_id in pgid2ids[i]:
            score = jac.get_sim_score(word_token.tokenize(data.patent_holders[p_id]), word_token.tokenize(target_string))
            if score > threshould:
                final_list.append(data.patent_holders[p_id])
    return final_list

## 1.2 - Declarations

In [7]:
# create a whitespace tokenizer
ws_tok = sm.WhitespaceTokenizer()

# create a qgram tokenizer using q = 3
qg3_tok = sm.QgramTokenizer(qval = 3)

In [8]:
# create a Jaccard similarity measure object
jac = sm.Jaccard()

# The Levenshtein measure treats the input strings as sequences of characters. Hence when using it we do not have to tokenize the two strings x and y.
lev = sm.Levenshtein()

# cosine distance
cos =  sm.Cosine()

# hamming distance
ham = sm.HammingDistance()

# 2(a) "United States Navy"

In [9]:
# load data_a
data_a = load_data(data_source = 'Patentholders.csv', column_name = 'patent_holders')
data_a.head(10)

,patent_holders
0,Signify Holding BV
1,ThyssenKrupp Airport Systems SA
2,Eurenco
3,EI du Pont de Nemours and Co
4,TRIATEX INTERNATIONAL AG
5,"THYSSENKRUPP TAILORED BLANKS GMBH, 47139 DUISB..."
6,Howe Wayne R
7,Carbone Lorraine Composants
8,Google LLC
9,MACOM Technology Solutions Holdings Inc


In [10]:
data_a = pruning_by_string(data = data_a, column_name = 'patent_holders', string = 'navy')
print(data_a.shape)
data_a.head(10)

(385, 1)


,patent_holders
0,NAVY SECRETARY OF UNITED STATE
1,NAVY SECRETARY OF
2,"UNTIED STATES OF AMERICA NAVY THE, Secretary of"
3,"NAVY GOVERNMENT OR United States, THE, Secreta..."
4,SECRETARY OF NAVY AS REPRESENTED BY UNITED STA...
5,"NAVY UNITED STATES THE, Secretary of"
6,UNITED STATES DEPARTMENT OF THE NAVY
7,"U S NAVY NAVY, Secretary of"
8,NAVY ISLAND PLYWOOD Inc
9,UNTIED STATES OF AMERICA REPRESENTED BY SECRET...


In [11]:
a_id2wstoken, a_id2qg3tokens, a_ids, a_id2gid = get_tokens(data = data_a, column_name = 'patent_holders')

In [12]:
a_pgid2ids = match_values_to_group(a_id2wstoken, index_list = a_ids, similarity_measure = jac, threshould = 0.5)

In [13]:
a_matched_list = find_matching_values(data = data_a, pgid2ids = a_pgid2ids, target_string = 'United States Navy', threshould = 0.2, word_token = ws_tok)
a_matched_list

['Navy United States Of American Re By Sec Of',
 'United States Department Of Navy',
 'United States, As Represented By Sercretary Of Navy',
 'United States, Navy Washington Dc (us), Secretary of']

# 2(b) "IBM"

In [14]:
# load data_b
data_b = load_data(data_source = 'Patentholders.csv', column_name = 'patent_holders')
data_b.head(10)

,patent_holders
0,Signify Holding BV
1,ThyssenKrupp Airport Systems SA
2,Eurenco
3,EI du Pont de Nemours and Co
4,TRIATEX INTERNATIONAL AG
5,"THYSSENKRUPP TAILORED BLANKS GMBH, 47139 DUISB..."
6,Howe Wayne R
7,Carbone Lorraine Composants
8,Google LLC
9,MACOM Technology Solutions Holdings Inc


In [15]:
data_b_test_1 = data_b[data_b['patent_holders'].str.startswith('i')]
data_b_test_2 = data_b[data_b['patent_holders'].str.startswith('I')]

frames = [data_b_test_1, data_b_test_2]

data_b = pd.concat(frames)
data_b.reset_index(drop = True, inplace = True)
print(data_b.shape)
data_b.head(10)

(330, 1)


,patent_holders
0,iAd Gesellschaft fur informatik Automatisierun...
1,ioGenetics LLC
2,iBOSS Inc
3,iRobot Corp
4,inVia Robotics LLC
5,iovation Inc
6,iDevices LLC
7,Intellectual Property Holdings LLC
8,Is5 Communications Inc
9,Ionis Pharmaceuticals Inc


In [16]:
b_id2wstoken, b_id2qg3tokens, b_ids, b_id2gid = get_tokens(data = data_b, column_name = 'patent_holders')

In [17]:
b_pgid2ids = match_values_to_group(b_id2wstoken, index_list = b_ids, similarity_measure = cos, threshould = 0.5)

In [18]:
b_matched_list_1 = find_matching_values(data = data_b, pgid2ids = b_pgid2ids, target_string = 'IBM', threshould = 0.15, word_token = qg3_tok)
b_matched_list_1

b_matched_list_2 = find_matching_values(data = data_b, pgid2ids = b_pgid2ids, target_string = 'International Business Machine', threshould = 0.6, word_token = qg3_tok)
b_matched_list_2

b_matched_list = list(set(b_matched_list_1 + b_matched_list_2))
b_matched_list

['ITESM', 'International Business Machines Corp', 'IBM Canada Ltd']

# 2(c) "Univ. of Calif."

In [19]:
# load data_c
data_c = load_data(data_source = 'Patentholders.csv', column_name = 'patent_holders')
data_c.head(10)

,patent_holders
0,Signify Holding BV
1,ThyssenKrupp Airport Systems SA
2,Eurenco
3,EI du Pont de Nemours and Co
4,TRIATEX INTERNATIONAL AG
5,"THYSSENKRUPP TAILORED BLANKS GMBH, 47139 DUISB..."
6,Howe Wayne R
7,Carbone Lorraine Composants
8,Google LLC
9,MACOM Technology Solutions Holdings Inc


In [20]:
data_c = pruning_by_string(data = data_c, column_name = 'patent_holders', string = 'univ')
data_c = pruning_by_string(data = data_c, column_name = 'patent_holders', string = 'calif')
print(data_c.shape)
data_c.head(10)

(5, 1)


,patent_holders
0,University of California
1,University of California Berkeley
2,University of Southern California USC
3,"CALIFORNIA A CORP OF, University of, Regents of"
4,California State University Long Beach


In [21]:
c_id2wstoken, c_id2qg3tokens, c_ids, c_id2gid = get_tokens(data = data_c, column_name = 'patent_holders')

In [22]:
c_pgid2ids = match_values_to_group(c_id2wstoken, index_list = c_ids, similarity_measure = jac, threshould = 0.5)

In [23]:
c_matched_list = find_matching_values(data = data_c, pgid2ids = c_pgid2ids, target_string = 'Univ. of Calif.', threshould = 0.2, word_token = qg3_tok)
c_matched_list

['University of California',
 'University of California Berkeley',
 'University of Southern California USC']